In [1]:
import openai

In [2]:
#!pip install praw

     -------------------------------------- 189.3/189.3 kB 3.8 MB/s eta 0:00:00


In [3]:
import praw

In [5]:
# This can be created when routed to https://www.reddit.com/prefs/apps and creating a redirect uri http://www.example.com/unused/redirect/uri
client_id = ''
client_secret = ''

In [27]:
# Hide this code block
#openai.api_key = 'Add your key here!'

In [6]:
reddit = praw.Reddit(client_id=client_id,
                    client_secret=client_secret,
                    user_agent = "sentiment analysis test"
                    )

In [12]:
subreddit_stocks = reddit.subreddit("stocks")

In [15]:
subreddit_stocks.accounts_active

655

In [17]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    # print top 2 comments per title submission
    counter = 0
    for comment in submission.comments:
        print(comment.body)
        counter+=1
        if counter == 2:
            break

/r/stocks US Election and Market Megathread
Stocks go down = Are we due for a correction?

Stocks go up = Are we overvalued?

The market is always nervous.
I've read so many different articles suggesting that markets will rally after we find out the results of the election, regardless of who wins, because the uncertainty goes away. Does anyone else find this difficult to comprehend? If we know for a fact that markets will go up by the end of this week, then what uncertainty is there in buying the index? Why wouldn't a guarantee of equity prices going up this week be already priced in today? They say that there's money on the sidelines waiting until election results come out to dive in - but if the victor of the election is irrelevant, why wait?
/r/Stocks Weekend Discussion Saturday - Nov 09, 2024
List of possible strangle targets for next week, using earnings as the catalyst for large price moves. If you're interested in an overnight play, make sure that the implied options move is les

In [20]:
def get_titles_and_comments(subreddit="stocks",limit=6,num_comments=3,skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}
    
    for counter,post in enumerate(subreddit.hot(limit=limit)):
        
        if counter < skip_first:
            continue
        counter += (1-skip_first)
        
        title_and_comments[counter] = ""
        # PRAW
        submission = reddit.submission(post.id)
        title = post.title
        # {0: "Title: Post Title \n\n Comments: \n\n"}
        title_and_comments[counter] +='Title: '+title+"\n\n"
        title_and_comments[counter] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                title_and_comments[counter] += comment.body+"\n"
                comment_counter += 1
            if comment_counter == num_comments:
                break
    return title_and_comments

In [21]:
titles_and_comments = get_titles_and_comments()

In [23]:
titles_and_comments

{1: 'Title: Reuters Exclusive: US ordered TSMC to halt shipments to China of chips used in AI applications\n\nComments: \n\nHi, you\'re on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.\n\n**To everyone commenting:**  Please focus on how this affects the stock market or specific stocks or it will be removed as being off-topic/political.\n\nIf you\'re interested in just politics, see our wiki on ["relevant subreddits"](https://www.reddit.com/r/stocks/wiki/index) and post to those Reddit communities instead without linking back here, thanks!\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/stocks) if you have any questions or concerns.*\nSeems like a lot of loss revenue.  I would have to look at the financials.\n\nBut Intel had like 27% of the revenue based in Chi

In [24]:
def create_prompt(title_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    return task+title_and_comments

In [26]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'

Title: Reuters Exclusive: US ordered TSMC to halt shipments to China of chips used in AI applications

Comments: 

Hi, you're on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.

**To everyone commenting:**  Please focus on how this affects the stock market or specific stocks or it will be removed as being off-topic/political.

If you're interested in just politics, see our wiki on ["relevant subreddits"](https://www.reddit.com/r/stocks/wiki/index) and post to those Reddit communities instead without linking back here, thanks!

*I am a bot, and this action was performed automatically. Please [contact the moderators of 

In [29]:
for key, title_with_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_with_comments)
    
    response = openai.Completion.create(engine='gpt-3.5-turbo-instruct',
                                      prompt=prompt,
                                      max_tokens=256,
                                      temperature=0,
                                      top_p=1.0
                                      )
    print(title_with_comments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print('-------------------------')

Title: Reuters Exclusive: US ordered TSMC to halt shipments to China of chips used in AI applications

Comments: 

Hi, you're on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.

**To everyone commenting:**  Please focus on how this affects the stock market or specific stocks or it will be removed as being off-topic/political.

If you're interested in just politics, see our wiki on ["relevant subreddits"](https://www.reddit.com/r/stocks/wiki/index) and post to those Reddit communities instead without linking back here, thanks!

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/stocks) if you have any questions or concerns.*
Seems like a lot of loss revenue.  I would have to look at the financials.

But Intel had like 27% of the revenue based in China, prior to the tal